<h1>Assignment 2</h1>

In [239]:
import os
import pandas as pd
import numpy as np

LINKS_PATH = os.path.join(os.getcwd(), 'movie', 'links.csv')
TAGS_PATH = os.path.join(os.getcwd(), 'movie', 'tags.csv')
MOVIES_PATH = os.path.join(os.getcwd(), 'movie', 'movies.csv')
RATINGS_PATH = os.path.join(os.getcwd(), 'movie', 'ratings.csv')

def load_data(path):
    return pd.read_csv(path)

<h4>Loading data</h4>

In [240]:
ratings = load_data(RATINGS_PATH)

In [241]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


<h2>Part A</h2>

<p>1. The first aggregation approach is the average method. The main idea behind this
approach is that all members are considered equals. So, the rating of an item for a group
of users will be given be averaging the scores of an item across all group members.</p>

<p>2. The second aggregation method is the least misery method, where one member can act
as a veto for the rest of the group. In this case, the rating of an item for a group of users is
computed as the minimum score assigned to that item in all group members
recommendations.</p>

<p>First let's get helper functions for predicting movie ratings from the previous assignment and prepare data</p>

In [242]:
# Helper functions from the previous assignment

# min_common_percentage = 0.1
min_common_items = 2

# Implementation of Pearson correlation
def get_similarity_between_two_items(user1, user2):
    # Find common items
    common_items = user1.notna() & user2.notna()
    
    common_items_count = common_items.sum()
    if common_items_count == 0:
        return 0  # No common items, no correlation
    
    # We implement a treshold to avoid meaningless correlations
    # Approach 1: Common items percentage
    # total_items_user1 = user1.count()
    # total_items_user2 = user2.count()
    # common_percentage_user1 = common_items_count / total_items_user1
    # common_percentage_user2 = common_items_count / total_items_user2
    # if common_percentage_user1 < min_common_percentage or common_percentage_user2 < min_common_percentage:
    #     return 0  # Not enough common items for a meaningful correlation

    # Approach 2: Common items count
    if common_items_count < min_common_items:
        return 0  # Not enough common items for a meaningful correlation
    
    # Get the common items
    user1_common = user1[common_items]
    user2_common = user2[common_items]
    
    # Pearson correlation requires at least 2 common items
    if len(user1_common) < 2:
        return 0 
    
    # Calculate the Pearson correlation coefficient
    correlation = user1_common.corr(user2_common)
    
    if np.isnan(correlation):
        return 0  # Handle NaN values
    
    return max(correlation, 0)  # Return a non-negative correlation

def predict_rating(user_id, movie_id, ratings_by_users):
    # If the user has already rated the movie, return the known rating
    if not np.isnan(ratings_by_users.loc[user_id, movie_id]):
        return ratings_by_users.loc[user_id, movie_id]

    # Get the users who rated the movie
    users_who_rated = ratings_by_users[ratings_by_users[movie_id].notna()].index

    # Calculate the similarities and the weighted ratings
    similarities = [get_similarity_between_two_items(ratings_by_users.loc[user_id], ratings_by_users.loc[other_user_id]) for other_user_id in users_who_rated]
    weighted_ratings = [similarity * (ratings_by_users.loc[other_user_id, movie_id] - ratings_by_users.loc[other_user_id].mean()) for other_user_id, similarity in zip(users_who_rated, similarities)]

    # If no one else rated the movie, return the mean rating of the user
    if sum(similarities) == 0:
        return ratings_by_users.loc[user_id].mean()

    # Return the weighted average rating, ensuring it is within the range of 0 - 5
    return max(min((sum(weighted_ratings) / sum(similarities)) + ratings_by_users.loc[user_id].mean(), 5), 0)


In [243]:
# Copying the ratings dataframe to a new dataframe for further processing
movie_ratings = ratings.copy()

In [244]:
# Making a pivot table to get the ratings of each movie by each user
ratings_by_users = movie_ratings.pivot_table(index='userId', columns='movieId', values='rating', aggfunc='first')
ratings_by_users.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<p>Implementing average agregation and least misery approaches for a set of users<p>

In [245]:
def group_rating_prediction(userIds, all_users_ratings, strategy='average', useFullDataset=False):
    # Get only the data about the users we are interested in
    group_users_ratings = all_users_ratings.loc[userIds]

    # Remove the movies that no one has rated
    group_users_ratings = group_users_ratings.dropna(axis=1, how='all')

    # Dataset to use for prediction
    if(useFullDataset):
        dataset_for_prediction = all_users_ratings.copy()
    else:
        dataset_for_prediction = group_users_ratings.copy()

    # Fill the missing ratings of every user by predicting them
    for user_id in userIds:
        for movie_id in group_users_ratings.columns:
            group_users_ratings.loc[user_id, movie_id] = predict_rating(user_id, movie_id, dataset_for_prediction)
    
    if(strategy == 'average'):
        # Get the average rating for every movie
        movie_ratings_average = group_users_ratings.mean(axis=0)

    if(strategy == 'least_misery'):
        # Get the least misery rating for every movie
        movie_ratings_average = group_users_ratings.min(axis=0)

    # Sort the movies by their average rating
    movie_ratings_average = movie_ratings_average.sort_values(ascending=False)

    return movie_ratings_average


<p>Produce a group of 3 users, and for this group, show the top-10 recommendations, i.e.,
the 10 movies with the highest prediction scores that (i) the average method suggests,
and (ii) the least misery method suggest</p>

In [246]:
# Predictions for the given group of users
userIds = [1, 2, 3]
average_predictions = group_rating_prediction(userIds, ratings_by_users, strategy='average')
least_misery_predictions = group_rating_prediction(userIds, ratings_by_users, strategy='least_misery')

/home/dmitrii/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]


/home/dmitrii/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]


In [247]:
# Show the top 10 movies for the given group of users by average rating
average_predictions.head(10)

movieId
2851     4.649425
70946    4.649425
5181     4.649425
849      4.649425
5746     4.649425
5919     4.649425
6835     4.649425
7991     4.649425
3703     4.649425
4518     4.649425
dtype: float64

In [248]:
# Show the top 10 movies for the given group of users by least misery rating
least_misery_predictions.head(10)

movieId
7991     3.948276
3703     3.948276
5919     3.948276
5764     3.948276
5746     3.948276
26409    3.948276
849      3.948276
5181     3.948276
2851     3.948276
1587     3.948276
dtype: float64

<p> The ratings are the same for all of the top 10 suggestions. This is caused by the fact that we used the specified group of users to predict missing movie values. There were a lot of movies rated only by one user and in this case the prediction fucntion returns the mean rating of the user<p>

<p>To avoid this kind of situation we can use the whole dataset to predict the missing movie ratings(since we use weighted ratings based on person correlation they should be accurate)</p>

<p> Let's do it </p>

In [249]:
# Using the full dataset for prediction
userIds = [1, 2, 3]
average_predictions = group_rating_prediction(userIds, ratings_by_users, strategy='average', useFullDataset=True)
least_misery_predictions = group_rating_prediction(userIds, ratings_by_users, strategy='least_misery', useFullDataset=True)


/home/dmitrii/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/dmitrii/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/dmitrii/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/dmitrii/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [250]:
average_predictions.head(10)

movieId
5181     4.649425
6835     4.649425
5746     4.649425
1927     4.644784
919      4.633701
80489    4.565617
954      4.531133
7899     4.482759
5764     4.482759
3147     4.475918
dtype: float64

In [251]:
least_misery_predictions.head(10)

movieId
1927     4.204374
80489    4.196850
2116     4.077340
4518     4.026903
5746     3.948276
5764     3.948276
6835     3.948276
7899     3.948276
5181     3.948276
3147     3.928063
dtype: float64

<p>As we can see now the ratings look better, but it required more computing to produce the results</p>

<h2>Part B</h2>

Define a way for counting the disagreements between the users in a group, and propose a method that takes disagreements into account when computing suggestions for the group

We propose two ways of computing suggestions with considering disagreements. One item-based and one user-based. The item-based takes the disagreements for each item rating into account, so how diverse the ratings for this movie is, the more diverse the less weight it will have when predicting. The user-based considers the disagreements between each pair of users for all items and determine a disagreement factor for each user, so that defines how much do their ratings differ from other users, the bigger this factor the less weight will they have when predicting the items.

<h3>User-based:</h3>
One way to capture disagreements is to consider the variance or diversity in user ratings within the group. The idea is to, calculate a disagreement matrix, for a group of users, that represents the differences in ratings between each pair of users. One way to measure this difference is by computing the squared differences between their ratings for common items. When computing recommendations for the group, we use a weighted averaging aggregiation, where the weights are inversely proportional to the disagreement between users. The rationale is that users who tend to disagree less should have more influence on the group recommendations.

In [252]:
def calculate_disagreement_matrix(ratings_by_relevant_user_matrix: pd.DataFrame) -> np.ndarray:
    # Get the number of users and movies
    number_of_users = ratings_by_relevant_user_matrix.shape[0]

    # Initialize the disagreement matrix
    disagreement_matrix = np.zeros((number_of_users, number_of_users))

    # Calculate the disagreement matrix
    for user1 in range(number_of_users):
        for user2 in range(user1 + 1, number_of_users):
            # Get the ratings of the two users
            user1_ratings = ratings_by_relevant_user_matrix.iloc[user1]
            user2_ratings = ratings_by_relevant_user_matrix.iloc[user2]

            # Get the common items
            common_items = user1_ratings.notna() & user2_ratings.notna()

            if common_items.sum() == 0:
                continue
            
            # Calculate the squared difference between the ratings
            disagreement = ((user1_ratings[common_items] - user2_ratings[common_items]) ** 2).sum() / common_items.sum()

            # Add the disagreement to the matrix
            disagreement_matrix[user1, user2] = disagreement
            disagreement_matrix[user2, user1] = disagreement

    return disagreement_matrix

def group_rating_prediction_with_disagreement(userIds, ratings_by_users):
    # Get only the data about the users we are interested in
    ratings_by_relevant_users = ratings_by_users.loc[userIds]

    # Remove the movies that no one has rated
    ratings_by_relevant_users = ratings_by_relevant_users.dropna(axis=1, how='all')
    
    # Calculate the disagreement matrix
    disagreement_matrix = calculate_disagreement_matrix(ratings_by_relevant_users)

    # weights will be inversely proportional to the sum of disagreements
    weights = 1 / (1 + disagreement_matrix.sum(axis=1))

    # aggregate the ratings of the users by calculating the weighted average
    weighted_average = pd.Series(index=ratings_by_relevant_users.columns)
    for movie_id in ratings_by_relevant_users.columns:
        weighted_average[movie_id] = (ratings_by_relevant_users[movie_id] * weights).sum() / weights.sum()
    
    # transform the ratings to the range of 0 - 5
    weighted_average = weighted_average * 5 / weighted_average.max()
    
    return weighted_average.sort_values(ascending=False)
    
group_rating_prediction_with_disagreement(userIds, ratings_by_users).head(10)

movieId
131724    5.000000
122882    5.000000
106782    5.000000
60756     5.000000
89774     5.000000
80906     5.000000
3578      4.861538
333       4.861538
1704      4.500000
58559     4.500000
dtype: float64

<h3>Item-based:</h3>

As mentioned above, the item-based takes the disagreements for each item rating into account, so how diverse the ratings for this movie is.
So we calculate the variance of the different ratings by users for each movie and we take the inverse of this value so it becomes the weight of the movie. Then we aggregate the user ratings for each movie by taking the weighted average. The idea behind this approach is that the group rating will be more influenced by the average rating of the group members for movies where they tend to agree, and less influenced by the average rating of the group members for movies where they tend to disagree.

In [253]:
def group_rating_prediction_with_variance(userIds, ratings_by_users):
    # Get only the data about the users we are interested in
    ratings_by_relevant_users = ratings_by_users.loc[userIds]

    # Remove the movies that no one has rated
    ratings_by_relevant_users = ratings_by_relevant_users.dropna(axis=1, how='all')

    # Fill the missing ratings of every user by predicting them
    for user_id in userIds:
        for movie_id in ratings_by_relevant_users.columns:
            ratings_by_relevant_users.loc[user_id, movie_id] = predict_rating(user_id, movie_id, ratings_by_relevant_users)
    
    # Calculate the variance of the ratings of the group members for each movie
    variance = ratings_by_relevant_users.var(axis=0, skipna=True)
    
    # weights will be inversely proportional to the variance
    movies_weights = 1 / (1 + variance)

    # aggregate the ratings of the users by calculating the weighted average
    weighted_average = pd.Series(index=ratings_by_relevant_users.columns)
    for movie_id in ratings_by_relevant_users.columns:
        weighted_average[movie_id] = (ratings_by_relevant_users[movie_id].sum() * movies_weights[movie_id]) / len(userIds)

    # transform the ratings to the range of 0 - 5
    weighted_average = weighted_average * 5 / weighted_average.max()
    return weighted_average.sort_values(ascending=False)

group_rating_prediction_with_variance(userIds, ratings_by_users).head(10)

    

/home/dmitrii/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/dmitrii/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


movieId
6835     5.000000
4518     5.000000
7991     5.000000
2851     5.000000
70946    5.000000
5746     5.000000
5181     5.000000
5919     5.000000
3703     5.000000
7899     4.461538
dtype: float64